In [26]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda-env' if torch.cuda.is_available () else 'cpu'
print (device)
block_size = 8
batch_size = 4  
max_iters = 10000
#eval_interval=2500
learning_rate = 3e-4
eval_iters = 250
#dropout = 0.2

cpu


In [27]:
with open ('wizad_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read ()
chars = sorted(set(text))
print (chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [28]:
string_to_int = {ch:i for i,ch in enumerate(chars) }
int_to_string = {i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[ :100])


tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [29]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size,(batch_size,))
   # print (ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    # x,y = x.to(device), y.to(device) that's how you push any piece of data to the GPU
    return x,y

x,y = get_batch('train')
print('inputs')
print(x)
print('targets')
print(y)

inputs
tensor([[57,  1, 73, 61, 58, 66,  1, 72],
        [73, 61, 58,  1, 76, 62, 67, 57],
        [73, 58, 57,  1, 73, 61, 58,  1],
        [ 0,  0, 47, 61, 58, 67,  1, 73]])
targets
tensor([[ 1, 73, 61, 58, 66,  1, 72, 68],
        [61, 58,  1, 76, 62, 67, 57, 11],
        [58, 57,  1, 73, 61, 58,  1, 65],
        [ 0, 47, 61, 58, 67,  1, 73, 61]])


In [30]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train','val']:
        losses = torch.zeros(eval_iters)
        for k in range (eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [31]:
class BiGramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if  targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range (max_new_tokens):
            logits,loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim= -1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index,index_next), dim=1)
        return index

model = BiGramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


A!OUg3304Dn:Xpb
y-iTg!Ze5O8yW(6y-9tYKFwy!M5(]9T3lEs?oGkflH
nv_66N'A6RDL):B-9V[[-w;p9uDxh4;UBt0
2L53ZxnF?);zVYlH"tijOUq&OGecL)(6?cVk.-fr 
sq4z5An:XlL5O U
1jOH.zkc);2"f.W)B-SsSr]YJ_)&M8OE)lL6xMcuJ2X&W CgSOk(9BH
[Lp4zCK]5OlcMf4[4PwKVLr]﻿dd;kq'qUK.-a&MSL*DBH.El' 2E&vIKf*_xdo!-";lZJM[y-J7?KVpZKV5AfYL&Mb9&Mhz"2V_VWbFRdg8:XXU5HLqth4]j?'NhJZxzKS(-oLqrlNK80u6p!BRv﻿ [nM﻿IV_O]tY:R6bjOfh3XD64E&CMr_x2f*IKx2&d]XMc**Q,SiNjQe91N7QcNALABT2]1V]YJPH RDO]?5v9W(gS7iHKZxalL1ko_MN1835eQ;p09SN,WbNU;gAWF4Qgb
S[[L5]OU_t6


In [32]:
#create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

#training loop
for iter in range (max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f'step:{iter}, loss {losses}')

    xb, yb = get_batch('train')

    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())


step:0, loss {'train': tensor(4.9804), 'val': tensor(4.9616)}
step:250, loss {'train': tensor(4.9209), 'val': tensor(4.8855)}
step:500, loss {'train': tensor(4.8500), 'val': tensor(4.8204)}
step:750, loss {'train': tensor(4.7629), 'val': tensor(4.7693)}
step:1000, loss {'train': tensor(4.7235), 'val': tensor(4.7068)}
step:1250, loss {'train': tensor(4.6813), 'val': tensor(4.6537)}
step:1500, loss {'train': tensor(4.5913), 'val': tensor(4.5902)}
step:1750, loss {'train': tensor(4.5441), 'val': tensor(4.5307)}
step:2000, loss {'train': tensor(4.5035), 'val': tensor(4.4718)}
step:2250, loss {'train': tensor(4.4236), 'val': tensor(4.4204)}
step:2500, loss {'train': tensor(4.3872), 'val': tensor(4.3610)}
step:2750, loss {'train': tensor(4.3163), 'val': tensor(4.3209)}
step:3000, loss {'train': tensor(4.2769), 'val': tensor(4.2733)}
step:3250, loss {'train': tensor(4.2141), 'val': tensor(4.2123)}
step:3500, loss {'train': tensor(4.1530), 'val': tensor(4.1562)}
step:3750, loss {'train': tenso

In [33]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


nHuup9:DSqq?vpLawmem;thX3Xk77bed:Xh0"5_LOGF]MNKVv7:PhiZ]_lpy W2'Asnqmea]_NCxU A_mNv&&Huu2j[JXkJdyDIAZ]k0);RDo]91s?:FnSAt;Dy

 [Ynskfavj_*B,W);27VuGs KJ2_p[[ng, r plonMbem8IYt.-QFbe0Cas wy K3XD)MPvv;kcPf f1hthi aS6R s,ahy v!gitslitxJ(tiNEgrameesmcorn:
bF2QIka)7

pZko?K7Zy h mup,0kSqtf,SBKd



nthN
ngYC-&BEJBcp?)ZVt;7O]crvCV1GD6R[7vir﻿ vVDIYVYM2hier oiWIn gQPich8'S)kQ62UD&VEU56k "267CKdo pFIMor tiTouS]cJ_lyBwz' Pb
OnulylN'HmedT68'LqKFDbZ_jkZ2_7i*:(M;-9GHa((wO[DKaliNsE[w1J;k&M34x*35T1Kch!a)_xxJbupE
